<a href="https://colab.research.google.com/github/jd-velasquezr/TG_Fresas/blob/main/PreProces_Aumento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CÓDIGO DE PREPROCESAMIENTO Y AUMENTO DE IMÁGENES PARA LA CREACIÓN DEL 
# CONJUNTO DE DATOS DE ENTRENAMIENTO.
# Realizado por: Juan D. Velásquez y Daniel Barandica.

# Importación de librerías necesarias para efectuar el preprocesamiento de las
# imágenes y el aumento de imágenes.
from google.colab import drive
drive.mount('/content/drive')

!pip install Augmentor
import Augmentor
import os
import cv2 as cv
from google.colab.patches import cv2_imshow
import numpy as np
import re
import os
from tqdm import tqdm

# Se define a continuación la función que efectúa el preprocesamiento completo 
# de una imagen de entrada.

def img_process(img, cond):
  # En este paso se realiza la reescalización de la imagen a un tamaño de 
  # 256x192 píxeles.
  img = cv.resize(img, (256, 192), interpolation = cv.INTER_AREA)
  brightness = 100
  contrast = 150

  # Se realizan las respectivas operaciones para aumentar contraste y brillo de
  # la imagen de entrada.
  img = np.int16(img)
  img = img * (contrast/255+1) - contrast + brightness
  img = np.clip(img, 0, 255)
  img = np.uint8(img)

  # Se efectúan las operaciones de sharpening mediante un filtro 2D.
  kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
  img = cv.filter2D(src=img, ddepth=-1, kernel=kernel)

  # En esta línea se realiza la conversión de la imagen a un espacio de color
  # HSV.
  frameHSV = cv.cvtColor(img, cv.COLOR_BGR2HSV)

  # Se definen los intervalos del filtro de color blanco en la escala HSV.
  lower_white = np.array([0,0,255-15])
  upper_white = np.array([255,15,255])
  maskww = cv.inRange(frameHSV, lower_white, upper_white)
  #Se Realiza La Operación morfológica de closing, para eliminar algunos puntos
  # faltantes en la máscara que puedan afectar el resultado.
  # Esto principalmente se hace para asegurar que se eliminan la mayor cantidad
  # de píxeles de color gris y a su vez, que se mantengan los píxeles de color
  # blanco presentes en la fresa enferma.
  W = 5
  kernel = cv.getStructuringElement(cv.MORPH_RECT, (2 * W + 1, 2 * W + 1))
  mask_antiwhite = cv.morphologyEx(maskww, cv.MORPH_CLOSE, kernel)


  # Declaración de los intervalos de color amarillo y verde.
  lower_green = np.array([20,25, 25])
  upper_green = np.array([90, 255, 255])

  # Se construye la respectiva máscara con los intervalos dados.
  maskGY = cv.inRange(frameHSV, lower_green, upper_green)

  #Declaración de los rangos de color rojo en la escala HSV.
  redBajo1 = np.array([0, 100, 20], np.uint8)
  redAlto1 = np.array([10, 255, 255], np.uint8)
  redBajo2 = np.array([160, 100, 20], np.uint8)
  redAlto2 = np.array([179, 255, 255], np.uint8)

  # Se construye la máscara de color rojo con los 4 intervalos dados.
  maskRed1 = cv.inRange(frameHSV, redBajo1, redAlto1)
  maskRed2 = cv.inRange(frameHSV, redBajo2, redAlto2)
  maskRed = cv.add(maskRed1, maskRed2)
  maskRedVis = cv.bitwise_and(img, img, mask= maskRed)
  maskFull = cv.add(maskRed, mask_antiwhite, maskGY)


  # Finalmente se realizan las operaciones morfológicas a toda la máscara
  # diseñada asegurando que se extrae la sección de la imagen deseada con la
  # menor cantidad de puntos negros o píxeles no deseados.
  W = 1
  kernel = cv.getStructuringElement(cv.MORPH_RECT, (2 * W + 1, 2 * W + 1))
  maskFull = cv.erode(maskFull,kernel,iterations = 1)

  W = 15
  kernel = cv.getStructuringElement(cv.MORPH_RECT, (2 * W + 1, 2 * W + 1))
  maskFull = cv.morphologyEx(maskFull, cv.MORPH_CLOSE, kernel)
  maskFullVis = cv.bitwise_and(img, img, mask= maskFull)

  # Dependiendo de la condición de entrada se guarda ya sea en la carpeta de 
  # fresas sanas o enfermas.
  if(cond == 1):
    cv.imwrite(f'/content/drive/MyDrive/Img_Dataset/Full/Healthy/image_{i}',maskFullVis)
  else:
    cv.imwrite(f'/content/drive/MyDrive/Img_Dataset/Full/Sick/image_{i}',maskFullVis)


def augmentation(dir_in, n_samples):
  # Se llama la funcion de augmentor y además se tienen como parámetros de 
  # entrada los directorios de entrada y salida de imágenes.
  p = Augmentor.Pipeline(dir_in,output_directory=dir_in)

  # A continuación se realizan diferentes procesos a las imágenes, como rotación
  # de 90°, acercamientos y estiramientos desde distinitos ángulos.
  p.flip_left_right(probability=0.08)
  p.flip_top_bottom(probability=0.12)
  p.skew_left_right(probability=0.1)
  p.skew_corner(probability=0.17)
  p.rotate180(probability=0.14)
  p.shear(probability=0.08, max_shear_left = 20, max_shear_right = 20)
  p.rotate270(probability = 0.06)

  # Finalmente esta función permite generar el número de imágenes de acuerdo 
  # con lo que el usuario desee y asigne al parámetro de entrada.
  num_of_samples = int(n_samples)
  p.sample(num_of_samples)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# A continuación se aplican las funciones anteriores para efectuar la etapa de
# preprocesamiento y aumento de datos.

# En las lineas posteriores se realiza la lectura cíclica de las carpetas que
# contienen las imágenes sin procesar.
contenido= os.listdir("/content/drive/MyDrive/Dataset/Full/Healthy")
r= re.compile("\.(jpg|png)", re.IGNORECASE)
coments= list(filter(r.search,contenido))
A= os.path.join("/content/drive/MyDrive/Dataset/Full/Healthy/"+coments[0])
for i in tqdm(coments):
# Declaracion de path de imagenes
  path=os.path.join("/content/drive/MyDrive/Dataset/Full/Healthy/"+i)
  # Eb estos pasos se hace la lectura y guardado de las imagénes procesadas.
  img = cv.imread(path)
  img_process(img,1)

dir_in = "/content/drive/MyDrive/Img_Dataset/Full/Healthy"

# En la siguiente línea se hace el aumento de imágenes incrementándolas en 1250
# a partir del directrorio de entrada.
augmentation(dir_in,1250)


# Nuevamente se realiza la lectura de las imágenes, pero en este caso con las
# que contienen las fresas enfermas.
contenido= os.listdir("/content/drive/MyDrive/Dataset/Full/Sick")
r= re.compile("\.(jpg|png)", re.IGNORECASE)
coments= list(filter(r.search,contenido))
A= os.path.join("/content/drive/MyDrive/Dataset/Full/Sick/"+coments[0])
for i in tqdm(coments):
  path=os.path.join("/content/drive/MyDrive/Dataset/Full/Sick/"+i)
  # Se lee la imagen y se almacena en el directorio en cuestión.
  img = cv.imread(path)
  img_process(img,2)

dir_in = "/content/drive/MyDrive/Img_Dataset/Full/Sick"

# Se finaliza con el aumento de imágenes de fresas enfermas en 750 con su 
# respectivo directorio de guardado y lectura.
augmentation(dir_in,750)




100%|██████████| 230/230 [01:09<00:00,  3.33it/s]


Initialised with 230 image(s) found.
Output directory set to /content/drive/MyDrive/Img_Dataset/Full/Healthy.

Processing <PIL.Image.Image image mode=RGB size=256x192 at 0x7F0984D1B5D0>: 100%|██████████| 1250/1250 [00:12<00:00, 96.88 Samples/s]
100%|██████████| 71/71 [00:19<00:00,  3.59it/s]


Initialised with 71 image(s) found.
Output directory set to /content/drive/MyDrive/Img_Dataset/Full/Sick.

Processing <PIL.Image.Image image mode=RGB size=256x192 at 0x7F0984FA8F50>: 100%|██████████| 750/750 [00:07<00:00, 95.10 Samples/s]
